In [1]:
import os
import sys
import json
import torch
import importlib
import matplotlib.pyplot as plt
import pandas as pd
# Get the current working directory
scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

import src.utils
importlib.reload(src.utils)
from src.utils import calculate_sum_rate_sc

import src.sc_wmmse
importlib.reload(src.sc_wmmse)
from src.sc_wmmse import WMMSE_alg_sc

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Update of Uk, Wk, Vk
def update_Uk(A, U, V, H, K, Nr):
    for k in range(K):
        A_item = np.zeros((Nr,Nr),complex)
        for m in range(K):
            A_item += H[k].dot(V[m]).dot(V[m].conjugate().T).dot(H[k].conjugate().T)
        A[k] = ((sigma**2)/PT) * np.sum([np.trace(np.dot(V[i],V[i].conjugate().T)) for i in range(K)])*np.eye(Nr) + A_item
        U[k] = np.linalg.inv(A[k]).dot(H[k]).dot(V[k])
    return U

def update_Wk(U, V, W_old, W, K, d):
    for k in range(K):
        E = np.eye(d) + 0j*np.eye(d) -(U[k].conjugate().T).dot(H[k]).dot(V[k])
        W_old[k] = W[k]
        W[k] = np.linalg.inv(E)
    return W, W_old

def update_Vk(V, W, U, H, K, Nt, sigma):
    for k in range(K):
        B_item = np.zeros((Nt,Nt),complex)
        for m in range(K):
            B_item += (H[m].conjugate().T).dot(U[m]).dot(W[m]).dot(U[m].conjugate().T).dot(H[m])
        B = B_item + sigma**2/PT*np.sum([np.trace(U[i].dot(W[i]).dot(U[i].conjugate().T)) for i in range(K)])*np.eye(Nt)
        V[k] = np.linalg.inv(B).dot(H[k].conjugate().T).dot(U[k]).dot(W[k])
    #Scale
    alpha = np.sqrt(PT/np.sum([np.trace(np.dot(V[i],V[i].conjugate().T)) for i in range(K)]))
    for k in range(K):
        V[k] = alpha * V[k]   
    return V


K = 10
PT = 100
sigma = 1
epsilon = 1e-5
d = 2
Nr = 2
Nt = 4
Imax = 100  #The maximum number of iterations
err = float("inf")

rate=[]

for num_test in range(1):
    #initial H
    mean1 = np.zeros(Nr)
    cov1 = np.eye(Nr)
    data = np.zeros((Nr,Nt))+1j*np.zeros((Nr,Nt))
    H = np.zeros((K,Nr,Nt))+1j*np.zeros((K,Nr,Nt))
    for i in range(K):
        data1 = np.random.multivariate_normal(mean1,cov1,Nt)
        data2 = np.random.multivariate_normal(mean1,cov1,Nt)
        data[0,:] = data1[:,0]+1j*data1[:,1]
        data[1,:] = data2[:,0]+1j*data2[:,1]
        H[i,:,:] = data; 
    
    #initial A、U、W, W_old
    A = []
    U = []
    W = []
    V = []
    V_init = []
    W_old = []
    for k in range(K):
        A.append(np.zeros((Nr,Nr),complex))
        U.append(np.zeros((Nr,d),complex))
        W.append(np.zeros((d,d),complex))
        W_old.append(np.eye(d)+complex("inf"))
        #V.append( np.sqrt(1/2)*(np.random.randn(Nt,Nr)+1j*np.random.randn(Nt,Nr)) )
        V.append(np.conj(np.matmul(np.linalg.pinv(np.matmul(H[k],np.conj(H[k]).T)),H[k])).T)
        V_init.append(np.conj(np.matmul(np.linalg.pinv(np.matmul(H[k],np.conj(H[k]).T)),H[k])).T)
        
    #Scale V
    alpha = np.sqrt(PT/np.sum([np.trace(np.dot(V[i],V[i].conjugate().T)) for i in range(K)]))
    for k in range(K):  #Scale V_k
        V[k] = alpha * V[k]           
    
    iteration = []
    sum_rate = []
    t = 0
    #while (err > epsilon) or t > Imax:
    while  t < 100:
        U = update_Uk(A, U, V, H, K, Nr)
        W, W_old = update_Wk(U, V, W_old, W, K, d)
        V = update_Vk(V, W, U, H, K, Nt, sigma)
        #err = abs(np.sum(np.fromiter([np.log(np.linalg.det(W[i])) for i in range(K)],complex)) -  np.sum(np.fromiter([np.log(np.linalg.det(W_old[i])) for i in range(K)],complex)))
        sum_rate.append(np.sum(np.fromiter([np.log2(np.linalg.det(W[k])) for k in range(K)],complex)))
        t += 1
        iteration.append(t) 
        
    rate.append(sum_rate[t-1])

# plt.plot(iteration,sum_rate)
# plt.xlabel("iteration")
# plt.ylabel("rate")
# plt.show()  
    
# print(np.array(rate).mean())
# sum_rate 

/Users/Ali/miniconda3/envs/Deep-Unfolding-NN/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/Ali/miniconda3/envs/Deep-Unfolding-NN/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [3]:
num_users_sc = 10
n_tx_sc = 4
n_rx_sc = [2] * num_users_sc
num_streams_sc = [2] * num_users_sc
P_T_sc = 100
sig_k_sc = [1] * num_users_sc
d_sc = [2] * num_users_sc
alpha_sc = [1] * num_users_sc
max_iter_alg_sc = 500
tol_alg_sc = 1e-3
H_dict_sc = {}

for k in range(num_users_sc):
    H_dict_sc[str(k)] = torch.from_numpy(H[k])

V_init_sc = {}
for k in range(num_users_sc):
    V_init_sc[str(k)] = torch.from_numpy(V_init[k])

def proj_power(V):
    alph = torch.sqrt(torch.tensor(P_T_sc)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(num_users_sc)])))
    V_proj = {str(k): alph * V[str(k)] for k in range(num_users_sc)}
    return V_proj

V_proj_sc = proj_power(V_init_sc)

w = WMMSE_alg_sc(num_users_sc, n_tx_sc, n_rx_sc, H_dict_sc, P_T_sc, sig_k_sc, d_sc, alpha_sc, max_iter_alg_sc, tol_alg_sc)

V_l_sc, U_l_sc, W_l_sc = w.algorithm(V_proj_sc)

r_l_sc = []
for i in V_l_sc:
    r_l_sc.append(calculate_sum_rate_sc(H_dict_sc, i, alpha_sc, sig_k_sc))
# plt.plot(r_l_sc)

W_l_l = []
for i in W_l_sc:
    val1 = 0
    for k in range(num_users_sc):
        val1 += torch.log2(torch.linalg.det(i[str(k)]))
    W_l_l.append(val1)
# plt.plot(W_l_l)

/var/folders/35/f3dbnk6d3ts1993_z878dt5w0000gn/T/ipykernel_90768/2740033701.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  alph = torch.sqrt(torch.tensor(P_T_sc)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(num_users_sc)])))
/Users/Ali/Projects/Deep-Unfolding-NN/src/sc_wmmse.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  alph = torch.sqrt(torch.tensor(self.PT)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(self.K)])))


In [4]:
sum_rate

[(5.629645336386401-5.243457830763763e-17j),
 (8.402395643878258+1.0903849793112681e-16j),
 (12.203137456115867-2.2703782927167358e-14j),
 (15.786774047278273+1.530093243433193e-14j),
 (19.123377966770594-2.4517781188901797e-15j),
 (21.922641608479033-2.523006307892291e-14j),
 (21.9972754238003-1.4884936530483412e-14j),
 (22.03268522382329+1.2250839810125489e-14j),
 (22.065713213856846+2.517431275614654e-14j),
 (22.09666355424025-8.650068256518236e-15j),
 (22.125790690071792+3.012977877463288e-14j),
 (22.153311028613505-1.3612835801365375e-14j),
 (22.179409623805604+4.762350270994133e-15j),
 (22.204245573294685-6.937576799061264e-15j),
 (22.227956396182226-1.4738453195450402e-14j),
 (22.250661599800452+1.6974321448446972e-14j),
 (22.27246559774964-5.13716438517266e-15j),
 (22.293460106428395+1.8410420591329523e-14j),
 (22.31372612045893-3.69253959157598e-15j),
 (22.333335546683735-1.894019447818885e-14j),
 (22.35235256027636+9.127221577926944e-15j),
 (22.370834733882432-3.4898196967376

In [5]:
r_l_sc

[tensor(8.4024+9.2143e-16j, dtype=torch.complex128),
 tensor(12.2031+4.0869e-15j, dtype=torch.complex128),
 tensor(15.7868-2.6430e-15j, dtype=torch.complex128),
 tensor(19.1234-1.2091e-14j, dtype=torch.complex128),
 tensor(21.9226+6.3055e-15j, dtype=torch.complex128),
 tensor(21.9973+2.2850e-14j, dtype=torch.complex128),
 tensor(22.0327+2.6961e-15j, dtype=torch.complex128),
 tensor(22.0657-1.2303e-14j, dtype=torch.complex128),
 tensor(22.0967-9.2217e-16j, dtype=torch.complex128),
 tensor(22.1258-4.5528e-15j, dtype=torch.complex128),
 tensor(22.1533+3.0245e-15j, dtype=torch.complex128),
 tensor(22.1794-5.2631e-15j, dtype=torch.complex128),
 tensor(22.2042-8.3098e-15j, dtype=torch.complex128),
 tensor(22.2280-1.2723e-14j, dtype=torch.complex128),
 tensor(22.2507-1.0932e-14j, dtype=torch.complex128),
 tensor(22.2725-1.8050e-14j, dtype=torch.complex128),
 tensor(22.2935-3.3022e-15j, dtype=torch.complex128),
 tensor(22.3137+1.5982e-14j, dtype=torch.complex128),
 tensor(22.3333+1.1744e-14j, 

In [6]:
W_l_l

[tensor(5.6296+2.9779e-17j, dtype=torch.complex128),
 tensor(8.4024-6.8481e-16j, dtype=torch.complex128),
 tensor(12.2031+5.8210e-15j, dtype=torch.complex128),
 tensor(15.7868+2.5714e-15j, dtype=torch.complex128),
 tensor(19.1234+2.0136e-14j, dtype=torch.complex128),
 tensor(21.9226-2.9315e-14j, dtype=torch.complex128),
 tensor(21.9973-2.2345e-14j, dtype=torch.complex128),
 tensor(22.0327+1.6036e-14j, dtype=torch.complex128),
 tensor(22.0657+4.0211e-14j, dtype=torch.complex128),
 tensor(22.0967-1.8584e-14j, dtype=torch.complex128),
 tensor(22.1258+1.5588e-14j, dtype=torch.complex128),
 tensor(22.1533+4.7362e-15j, dtype=torch.complex128),
 tensor(22.1794-1.4737e-14j, dtype=torch.complex128),
 tensor(22.2042-8.6553e-16j, dtype=torch.complex128),
 tensor(22.2280+1.3487e-14j, dtype=torch.complex128),
 tensor(22.2507+2.6113e-14j, dtype=torch.complex128),
 tensor(22.2725+2.0685e-14j, dtype=torch.complex128),
 tensor(22.2935-1.1764e-14j, dtype=torch.complex128),
 tensor(22.3137-2.2962e-14j, d